# Análisis Sintáctico (Parser)
---

## Funciones del Parser

* Comprobar que los tokens que le suministra el Scanner van ordenados según la especificación de la gramática del lenguaje a compilar
* Si no es así, dar los mensajes de error adecuados, pero continuar funcionando sin detenerse, hasta que se llegue al final del archivo de entrada
* Guia casi todo el proceso de la compilación. Esto es así porque por un lado va solicitando al Scanner los tokens y al mismo tiempo va dirigiendo el proceso de análisis semántico y generación de código intermedio (ambos procesos se les llama, traducción dirigida por la sintaxis)

![Fases de compilación](img/fase-parser.png)

* Generalmente, los Parsers obtienen un árbol teórico, árbol de análisis sintáctico (AAS) que permite expresar el orden de los lexemas según van apareciendo
* Si se utiliza el método de traducción dirigida por la sintaxis no se llega ni siquiera a plantearse la generación del árbol ya que el Parser realizará las acciones semánticas e incorporará los métodos para realizar la generación de código intermedio y avisará de errores y su recuperación
* Es decir, el Parser hará las funciones de las dos fases siguientes (analizador semántico y generación de código intermedio)
* Si se le da la oportunidad a la creación del AAS, recorriéndolo es posible crear una representación intermedia del programa fuente, ya sea en forma de árbol sintáctico abstracto o en forma de programa en un lenguaje intermedio
* Para generar el Parser, se pueden utilizar dos técnicas:

| Técnica                                                  | Implementación | Eficiencia      |
| --                                                       | --             | --              |
| **a mano**                                               | difícil        | eficiente       |
| **mediante herramientas que lo generan automáticamente** | fácil          | menos eficiente |

## Especificación del Lenguaje (L)

* Para que un Parser funcione, se debe especificar formalmente L
* L debe ser representado con reglas únicas y bien formadas (GIC) de manera que el Parser funcione de una manera bien definida
* Primer paso para implementar un Parser es definir la GIC que debe ser capaz de analizar
* Una GIC es una especificación (formal) de un conjunto de strings válidos (programas)
* La GIC no explica el "significado" de las partes del programa
* Ejemplo: "Cobertorzinho"

```
<exp> ::= a 
<exp> ::= b
<exp> ::= virar(<exp>)
<exp> ::= costurar(<exp>,<exp>)
```

## Diseño de Gramáticas Independientes/Libre de Contexto (GIC)

* GIC es aquella en donde sus reglas de producción se pueden aplicar sin considerar el contexto del no-terminal
* GIC se define por tuplas de 4 elementos:

| Elementos  |                                                | Representación                        |
| --         | --                                             | --                                    |
| $\Sigma_T$ | conjunto finito de símbolos terminales         | negrita                               |
| $\Sigma_N$ | conjunto finito de símbolos no terminales      | palabras que comienzan con mayúsculas |
| $P$        | conjunto de reglas de reescritura o producción | ::= ó $\rightarrow$                   |
| $S$        | axioma inicial o símbolo distinguido           |                                       |

## Ejemplos

### Sintaxis del lenguaje de fechas (GIC no recursiva)

```
<date> ::= <d><d>/<d><d>/<d><d><d><d>
```

### Sintaxis del lenguaje de secuencia de números (GIC recursiva)

```
<real_number> ::= <digit_seq> . <digit_seq>
<digit_seq> ::= <d> | <d> <digit_seq>
<d> ::= 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9
```

### Sintaxis del lenguaje de expresiones aritméticas

```
E -> E + E | E * E | num | id | (E)
```

* Derivaciones para la cadena: id1 + id2 * id3

|                      |                                                                                 |
| --                   | --                                                                              |
| **Por la izquierda** | E -> E \* E -> E + E \* E -> id1 + E \* E -> id1 + id2 \* E -> id1 + id2 \* id3 |
| **Por la derecha**   | E -> E \* E -> E \* id3 -> E + E \* id3 -> E + id2 \* id3 -> id1 + id2 \* id3   |

* Se observa que la gramática es **ambigua**, para su implementación, es necesario evitar la ambigüedad

```
E -> E + T | T
T -> T * F | F
F -> id | F | (E)
```

## Gramática ambigua

* Cuando una GIC genera una palabra para la que hay más de un AAS se dice que es ambigua
* Debido a que una GIC de estas características permite que a partir del mismo código fuente se puedan obtener diferentes códigos intermedios, no es válido para construir un compilador
* Técnicas que aseguran que si una GIC cumple ciertas reglas, se sabrá con seguridad que es ambigua y por lo tanto no se podrá construir un compilador con ella. En algunos casos se podrá hacer una serie de modificaciones en la GIC que la convierta en no ambigua
* Si una GIC tiene alguna de estas características, se podrá afirmar que es ambigua:

|||
| -- | -- |
| **GIC con ciclos**                   | $S \rightarrow A$    $S \rightarrow a$    $A \rightarrow S$ |
| **GIC con alguna regla de la forma** | $E \rightarrow E ... E$                                     |
| **GIC con unas reglas que ofrezcan caminos alternativos entre dos puntos** | $S \rightarrow B$    $S \rightarrow C$    $B \rightarrow C$ |
| **Producciones recursivas en las que las variables no recursivas de la producción puedan derivar a la palabra vacía** | $S \rightarrow A B S \mid S$    $A \rightarrow a \mid \lambda$    $B \rightarrow b \mid \lambda$ |
| **Símbolos no terminales que puedan derivar a la palabra vacía y a la misma palabra de terminales, y que aparezcan juntas en la parte derecha de una regla o en alguna forma sentencial** | $A \rightarrow A B \mid a \mid \lambda$    $B \rightarrow b \mid a \mid \lambda$ |

## Tipos de análisis sintáctico y GICs

* Al derivar una secuencia de tokens, si existe más de un no terminal en una cadena de derivación se debe elegir cuál es el próximo no terminal que se va a expandir, es decir, cuál será reemplazado por su lado derecho de la producción
* Por ello se utilizan dos tipos de derivaciones que determinan con precisión cuál será el no terminal a tratar:

|                            |                                                                                                                                 |
| --                         | --                                                                                                                              |
| **Derivación a izquierda** | ocurre cuando siempre se reemplaza el primer no terminal que se encuentre en una cadena derivación leída de izquierda a derecha |
| **Derivación a derecha**   | ocurre cuando siempre reemplaza el último no terminal de la cadena de derivación leída de izquierda a derecha                   |

## Estrategias de análisis sintáctico

* Hay varios algoritmos de análisis sintáctico (incluso para las gramáticas ambiguas), pero su costo computacional es elevado -> $O(n^3)$
* Por lo que se debe modificar la GIC (si es necesario) para que se pueda utilizar un algoritmo de menor costo computacional -> $O(n)$
* Si se consigue eliminar la ambigüedad, se pueden utilizar dos estrategias:

|||
| -- | -- |
| **Análisis Sintáctico Descendente (ASD)** | produce una derivación por izquierda, que comienza en el axioma y finaliza con los terminales que forman la construcción analizada |
| **Análisis Sintáctico Ascendente (ASA)**  | utiliza una derivación a derecha, pero en orden inverso, esto es: la última producción aplicada en la derivación a derecha, es la primera producción que es "descubierta", mientras que la primera producción utilizada, la que involucra al axioma, es la última producción en ser "descubierta". En otras palabras, "reduce el árbol de análisis sintáctico" hasta llegar al axioma |

### Análisis Sintáctico Descendente (ASD)

* Se parte de la raíz del AAS y se van aplicando **reglas por la izquierda** para obtener una derivación por la izquierda del símbolo inicial
* Para saber la regla a aplicar, se van leyendo tokens de la entrada
* De esta manera se construye el AAS
* Recorriendo el árbol en profundidad, de izquierda a derecha, se tendrá en las hojas los tokens ordenados
* Las gramáticas de tipo LL(k) se pueden analizar en tiempo lineal por el método de análisis descendente
  * k: número de símbolos de entrada que es necesario conocer en cada momento para poder realizar el análisis

* Ejemplo: derivar la cadena $aabcdd$

$$S \rightarrow aST \mid b$$
$$T \rightarrow cT \mid d$$

| Cadena de derivación obtenida | Próxima producción a aplicar |
| --                            | --                           |
| $S$                           | $S \rightarrow aST$          |
| $aST$                         | $S \rightarrow aST$          |
| $aaSTT$                       | $S \rightarrow b$            |
| $aabTT$                       | $T \rightarrow cT$           |
| $aabcTT$                      | $T \rightarrow d$            |
| $aabcdT$                      | $T \rightarrow d$            |
| $aabcdd$                      | $accept$                     |

* Tipos de análisis:
  * **Análisis Sintáctico Descendente con retroceso** -> ASD con retroceso
  * **Análisis Sintáctico Descendente Predictivo** -> ASDP LL(1)

### Análisis Sintáctico Ascendente (ASA)

* Se parte de la palabra de entrada y se va construyendo el árbol a partir de las hojas para llegar a la raíz
* Si se recorre el árbol generado, se encontrarán los tokens ordenados
* Las gramáticas de tipo LR(k) se pueden analizar en tiempo lineal por el método de análisis ascendente

* Ejemplo: derivar la cadena $aabcdd$

$$S \rightarrow aST \mid b$$
$$T \rightarrow cT \mid d$$

1. Derivación a derecha:

| Cadena de derivación obtenida | Próxima producción a aplicar |
| --                            | --                           |
| $S$                           | $S \rightarrow aST$          |
| $aST$                         | $T \rightarrow d$            |
| $aSd$                         | $S \rightarrow aST$          |
| $aaSTd$                       | $T \rightarrow cT$           |
| $aaScTd$                      | $T \rightarrow d$            |
| $aaScdd$                      | $S \rightarrow b$            |
| $aabcdd$                      |                              |

2. Orden Inverso a la derivación por derecha

| Cadena de derivación obtenida | Próxima producción a aplicar |
| --                            | --                           |
| $aabcdd$                      | $S \rightarrow b$            |
| $aaScdd$                      | $T \rightarrow d$            |
| $aaScTd$                      | $T \rightarrow cT$           |
| $aaSTd$                       | $S \rightarrow aST$          |
| $aSd$                         | $T \rightarrow d$            |
| $aST$                         | $S \rightarrow aST$          |
| $S$                           | $accept$                     |

* Tipos de análisis:
  * **Análisis Sintáctico Ascendente con retroceso** -> ASA con retroceso
  * **Análisis Sintáctico Ascendente Predictivo** -> ASAP SLR

## Diagrama de sintaxis

* Son grafos dirigidos donde los elementos no terminales de la GIC aparecen como rectángulos y los terminales como círculos o elipses
* Todo diagrama de sintaxis se supone que tiene un origen y un destino, aunque no se dibujan (se supone que el origen está a la izquierda y el destino a la derecha)

![Diagrama IF](img/if.png)

* Ejemplo: $$Sentencia \rightarrow Identificador \, = \, Número$$

```
---->[Identificador]---->(=)---->[Número]---->
```

In [1]:
def secuencia():
    if token == Identificador: 
        token = getToken()
    elif token == IGUAL: 
        token = getToken()
    elif token == Numero: 
        token = getToken()
    else: 
        error()

* Ejemplo: $$Sentencia \rightarrow (Sentencia \, ; \,)^+$$

```
-----------------------------
|                           ^ 
v                           |
---->[Sentencia]---->(;) ------->
```

In [3]:
def secuencia():
    token = getToken();
    while token != EOF:
      sentencia()
      while (token != PUNTOCOMA):
        error(); 
        token = getToken()
      token = getToken()

## Tools(Parser(Languages(Program)))

* Un desarrollador está expuesto a un número creciente de tecnología de parsing:
  * (mini) Herramientas
    * Pretty Print (... a colores!!)
    * Autocompletado de código
    * AI (Copilot,  Tabnine, Kite)
  * Herramientas
    * Refactoring
    * Crítica de Código y CodeSmells
    * Source Base Project Analytics

* Por ejemplo: Como se implementa el formateo automático o los colores en el código fuente?

* Programa: al menos un lenguaje
* Lenguajes: Gramática (sintaxis+) y Semántica de operación (y denotativa)
* Parsers
  * "Analizan" la validez de un texto como programa
  * Consideran la gramática de un lenguaje
  * Se basan en el AST (abstract syntax trees)
* Herramientas
  * Analizar/Mantener/Documentar
  * Refactorizar/Optimizar
  * Migrar/Reescribir

## Herramientas

### CUP

* Constructor of Useful Parsers  
* Genera analizadores sintácticos LALR (LookAhead LR)
* Genera código Java y permite introducir acciones semánticas escritas en dicho lenguaje

* Declaraciones **JLex**:

```
import java_cup.runtime.Symbol;
%%
%full
%notunix
%cup
%%
[\r\n\t ]+ {/*prescindir de blancos*/}
"+" { return new Symbol (sym.SUMA); }
"-" { return new Symbol (sym.RESTA); }
"*" { return new Symbol (sym.MULTIPLICACION); }
"/" { return new Symbol (sym.DIVISION); }
"^" { return new Symbol (sym.POTENCIA); }
"(" { return new Symbol (sym.LPARENT); }
")" { return new Symbol (sym.RPARENT); }
"=" { return new Symbol (sym.RESULTADO); }
[0-9]+ {return new Symbol (sym.ENTERO, new Integer (yytext())); }
[^0-9\r\n\t \+\-\*"^"/]+ { System.out.println("Error léxico: "+ yytext() ); }
```

* Declaraciones **CUP**

```
// terminales y no terminales
terminal SUMA, RESTA,DIVISION,MULTIPLICACION;
terminal Integer ENTERO;
terminal POTENCIA, RESULTADO, LPARENT, RPARENT, SIGNO;
non terminal sesion, ecuacion;
non terminal Integer expresion;
precedence left RESTA, SUMA;
precedence left MULTIPLICACION, DIVISION;
precedence right SIGNO;
// gramática
sesion ::= ecuacion
         | ecuacion sesion;
ecuacion ::= expresion:E1
             {: System.out.println(E1.intValue()); :}
RESULTADO;
expresion ::= ENTERO:E1
              {:RESULT = new Integer (E1.intValue()); :}
            | expresion:E1 SUMA expresion:E2
              {:RESULT=new Integer( E1.intValue() + E2.intValue()); :}
            | expresion:E1 RESTA expresion:E2
              {:RESULT=new Integer( E1.intValue() - E2.intValue()); :}
            | expresion:E1 MULTIPLICACION expresion:E2
              {:RESULT=new Integer( E1.intValue() * E2.intValue()); :}
            | expresion:E1 DIVISION expresion:E2
              {:RESULT=new Integer(E1.intValue() / E2.intValue()); :}
            | LPARENT expresion:E1 RPARENT
              {:RESULT=new Integer(E1.intValue()); :}
            | RESTA expresion:E1
              {: RESULT=new Integer(0-E1.intValue()); :}
%prec SIGNO;
```

### ANTLR4

* ANother Tool for Language Recognition ("otra herramienta para reconocimiento de lenguajes")
* Proporciona un framework para construir scanners y parsers a partir de una gramática
* Se puede usar para la construcción de lenguajes, herramientas y frameworks
* Para una gramática, puede generar un parser que construya un árbol AST (árbol de sintaxis abstracta)
* [Sitio Web](https://tomassetti.me/antlr-mega-tutorial/)

```
grammar Expr;

prog:   (expr NEWLINE)*;
expr:   expr ('*'|'/') expr
    |   expr ('+'|'-') expr
    |   INT
    |   '(' expr ')'
    ;
NEWLINE : [\r\n]+;
INT     : [0-9]+;
```

```
$ antlr4 Expr.g4
$ javac Expr*.java
$ grun Expr prog -gui
100+2*34
^D
```

```
grammar Identificadores;

programa: identificador;
identificador : LETRA (LETRA | NUMERO)*;

NUMERO : [0-9]+;
LETRA : ([A-Z] | [a-z])+;

WS : [ \t\r\n]+ -> skip;
```

### Bison

* Es un programa generador de parsers de propósito general perteneciente al proyecto GNU
* Se usa normalmente acompañado de flex aunque los scanners se pueden también obtener de otras formas
* Convierte la descripción formal de un lenguaje, escrita como una GIC LALR, en un programa en C, C++, o Java que realiza parsing
* [Sitio Web Oficial](https://www.gnu.org/software/bison/)

### PLY

* Es una implementación en Python de lex y yacc, generadores de scanners y parsers respectivamente
* Se definen los patrones de los diferentes tokens que se desean reconocer, esto se hace a través de ER
* Se definen las producciones y acciones para formar la gramática a través de funciones

In [23]:
%%file robot.py
import ply.lex as lex
import ply.yacc as yacc

# definir tokens
tokens = ('COMIENZA', 'NORTE', 'SUR', 'ESTE', 'OESTE', 'FIN')

# definir patrones
t_COMIENZA = r'C'
t_NORTE    = r'N'
t_SUR      = r'S'
t_ESTE     = r'E'
t_OESTE    = r'O'
t_FIN      = r'F' 
t_ignore   = ' \t'

def t_newline(t):
    r'\n+'
    t.lexer.lineno += t.value.count("\n")

def t_error(t):
    print("Carácter ilegal '%s'" % t.value[0])
    t.lexer.skip(1)

# construir scanner
lexer = lex.lex()

robot = {'x': 0, 'y':0}

def p_programa(t):
    '''programa : COMIENZA instrucciones FIN 
                | COMIENZA FIN '''
    print(robot)

def p_instrucciones_lista(t):
    '''instrucciones    : instruccion instrucciones
                        | instruccion '''

def p_instruccion_norte(t):
    'instruccion : NORTE'
    robot['y'] += 1

def p_instruccion_sur(t):
    'instruccion : SUR'
    robot['y'] -= 1

def p_instruccion_este(t):
    'instruccion : ESTE'
    robot['x'] += 1

def p_instruccion_oeste(t):
    'instruccion : OESTE'
    robot['x'] -= 1
    
def p_error(t):
    print("Error sintáctico en '%s'" % t.value)

parser = yacc.yacc()
parser.parse('CSNNSOOONF')

Overwriting robot.py


In [24]:
!python robot.py

{'x': -3, 'y': 1}


### PetitParser

* Es un framework para construir parsers:
* Permite crear parsers eficientes en diferentes lenguajes de programación: C#, Clojure, Dart, Java, Kotlin, PHP, Python, Smalltalk, Swift y TypeScript
* [Sitio Web](https://petitparser.github.io/)

In [10]:
from petitparser import character as c

ident = c.letter() & (c.letter() | c.digit()).star()

# Análisis de cadenas:
id1 = ident.parse('yeah')
print(id1.value) # ['y', ['e', 'a', 'h']]

id2 = ident.parse('f12')
print(id2.value) # ['f', ['1', '2']]

id3 = ident.parse('123')
print(id3.message)  # letter expected
print(id3.position) # 0

print(ident.accept('foo')) # True
print(ident.accept('123')) # False

['y', ['e', 'a', 'h']]
['f', ['1', '2']]
letter expected
0
True
False
